In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,241 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,134 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [979 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-secur

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
house_data = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, ignoreLeadingWhiteSpace=True)
#Observe the need to use ignoreLeadingWhiteSpace=True, otherwise a leading whitespace will appear in the column names

# Show DataFrame
house_data.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
house_data.createOrReplaceTempView('house_data')


In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

spark.sql("""SELECT YEAR(date) as sale_year,
           ROUND(AVG(price), 2) as avg_price
    FROM house_data
    WHERE bedrooms = 4
    GROUP BY sale_year
    ORDER BY sale_year ASC
""").show()


+---------+---------+
|sale_year|avg_price|
+---------+---------+
|     2019| 300263.7|
|     2020|298353.78|
|     2021|301819.44|
|     2022|296363.88|
+---------+---------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms
#rounded to two decimal places?
spark.sql(""" SELECT YEAR(date_built) AS YEAR,
        ROUND(AVG(price), 2) AS Avg_Price
      FROM house_data
      WHERE bedrooms = 3 AND bathrooms = 3
      GROUP BY YEAR(date_built)
      ORDER BY YEAR
 """).show()




+----+---------+
|YEAR|Avg_Price|
+----+---------+
|2010|292859.62|
|2011|291117.47|
|2012|293683.19|
|2013|295962.27|
|2014|290852.27|
|2015| 288770.3|
|2016|290555.07|
|2017|292676.79|
+----+---------+



In [7]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
# Define your query
spark.sql("""SELECT YEAR(date_built) AS Year_Built,
            ROUND(AVG(price), 2) AS Avg_Price
        FROM house_data
        WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_lot >= 2000
        GROUP BY date_built
        ORDER BY date_built ASC
      """).show()


+----------+---------+
|Year_Built|Avg_Price|
+----------+---------+
|      2010|280447.23|
|      2011|281413.45|
|      2012|295858.68|
|      2013|295142.13|
|      2014|294195.13|
|      2015|291788.36|
|      2016| 287812.6|
|      2017|282026.59|
+----------+---------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places,
#where the homes are greater than or equal to $350,000?
#Although this is a small dataset, determine the run time for this query.
#https://colab.research.google.com/drive/1xEtgEaTkrSSzJ-6wdnhFsoaQZu76wjOs?authuser=1#scrollTo=daHsHUbSTzaj


start_time = time.time()

spark.sql("""SELECT AVG(price) AS avg_price,
            ROUND(AVG(view), 2) AS avg_view_rating
        FROM house_data
        WHERE price >= 350000
        GROUP BY view
    """).show()

# # Calculate and print the runtime
end_time = time.time()
runtime = end_time - start_time
print("Query runtime:", runtime, "seconds")





#print("--- %s seconds ---" % (time.time() - start_time))

+------------------+---------------+
|         avg_price|avg_view_rating|
+------------------+---------------+
| 403005.7709497207|            7.0|
| 788128.2083333334|           51.0|
|404673.29545454547|           15.0|
| 798684.8235294118|           54.0|
|399548.11891891895|           11.0|
| 397771.6512820513|           29.0|
| 750537.9411764706|           69.0|
|396964.50241545896|           42.0|
|         1072285.2|           87.0|
| 752861.1764705882|           73.0|
| 767036.6666666666|           64.0|
| 398867.5964912281|            3.0|
|          397862.0|           30.0|
| 401419.7451923077|           34.0|
|          791453.0|           59.0|
|398592.70658682636|            8.0|
|402124.62176165805|           28.0|
| 402022.6847826087|           22.0|
|1056336.7435897435|           85.0|
| 399586.5311004785|           16.0|
+------------------+---------------+
only showing top 20 rows

Query runtime: 1.1118435859680176 seconds


In [9]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("house_data")


In [10]:
# 8. Check if the table is cached.
is_cached = spark.catalog.isCached("house_data")

if is_cached:
    print("The table 'house_data' is cached.")
else:
    print("The table 'house_data' is not cached.")


The table 'house_data' is cached.


In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time_cached = time.time()


# Run the query on the cached table
query = """
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM house_data
    WHERE price >= 350000
    GROUP BY view
"""
result_cached = spark.sql(query)

end_time_cached = time.time()

# Determine the runtime for the cached query
runtime_cached = end_time_cached - start_time_cached
print(f"Cached Query Runtime: {runtime_cached} seconds")




Cached Query Runtime: 0.04361271858215332 seconds


In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
house_data.write.partitionBy("date_built").mode("overwrite").parquet("house_data")

In [13]:
# 11. Read the parquet formatted data.
hd_p=spark.read.parquet('house_data')

In [14]:
# 12. Create a temporary table for the parquet data.
house_data.createOrReplaceTempView('hd_p')


In [15]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time_parquet = time.time()

query = """
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM hd_p
    WHERE price >= 350000
    GROUP BY view
"""
result_parquet = spark.sql(query)

end_time_parquet = time.time()

# Determine the runtime for the Parquet query
runtime_parquet = end_time_parquet - start_time_parquet
print(f"Parquet Query Runtime: {runtime_parquet} seconds")




Parquet Query Runtime: 0.061580657958984375 seconds


In [16]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("house_data")


In [17]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("house_data")
if is_cached:
    print("house_data table is cached.")
else:
    print("house_data table is not cached.")

# Stop the Spark session
spark.stop()

house_data table is not cached.
